In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.graph_objs as go
from plotly.offline import iplot

In [ ]:
# Use .item() to convert from numpy array to dictionary for easier loading
track_data = np.load('tracks.npy', allow_pickle=True).item()
track_df = pd.DataFrame.from_records(track_data)
#track_df.head()

# Flatten points array for easier xyz extracting 
points = np.concatenate(track_data['points'])
print(points.shape)

track_xs = points[:,0]
track_ys = points[:,1]
track_zs = points[:,2]

In [ ]:
crthit_data = np.load('crthits.npy', allow_pickle=True).item()
crthit_df = pd.DataFrame.from_records(crthit_data)
#crthit_df.head()

In [ ]:
match_candidate_data = np.load('match_candidates.npy', allow_pickle=True).item()
match_candidate_df = pd.DataFrame.from_records(match_candidate_data)
print(match_candidate_df.head())

matched_tracks = match_candidate_df['track']
matched_crthits = match_candidate_df['crthit']
matched_track_ids  = [matched_tracks[i].id for i,_ in enumerate(matched_tracks)]
matched_crthit_ids = [matched_crthits[i].id for i,_ in enumerate(matched_crthits)]

In [ ]:
def plot_crt_tpc_matches(track_dataframe, crthit_dataframe, matched_track_ids, matched_crthit_ids):
    
    traces = []
    
    for index, track in track_dataframe.iterrows():
        
        track_points = track['points']
        track_id = track['id']
        
        track_xs = track_points[:,0]
        track_ys = track_points[:,1]
        track_zs = track_points[:,2]

        track_trace = go.Scatter3d(
            x = track_xs,
            # Flip y and z so that plotly plots the long axis as the z-direction
            y = track_zs,
            z = track_ys,
            mode='markers',
            marker=dict(size=1, color='lightslategrey'),
            name='Track ID ' + str(track_id),
            customdata=np.stack((track_xs, track_ys, track_zs), axis=-1),
            hovertemplate='<b>x</b>: %{customdata[0]:,.2f}<br>' +
                          '<b>y</b>: %{customdata[1]:,.2f}<br>' +
                          '<b>z</b>: %{customdata[2]:,.2f}',
                          showlegend=False)
        
        if track_id in matched_track_ids:
            track_trace.marker.color = 'red'
        
        traces.append(track_trace)
        
    crthit_id = crthit_dataframe['id']
    crthit_xs = crthit_dataframe['position_x']
    crthit_ys = crthit_dataframe['position_z']
    crthit_zs = crthit_dataframe['position_y']
    
    matched_crthit_mask = crthit_id.isin(matched_crthit_ids)
    marker_color = np.array(['lightslategrey'] * len(crthit_xs)).astype(object)
    marker_color[matched_crthit_mask] = 'red'
    
    crthit_trace = go.Scatter3d(
        x = crthit_xs,
        y = crthit_ys,
        z = crthit_zs,
        mode='markers',
        marker=dict(size=2, color=marker_color),
        customdata=np.stack((crthit_id, crthit_xs, crthit_ys, crthit_zs), axis=-1),
        #name=[f"CRT Hit ID {id}" for id in crthit_id],
        hovertemplate='<b>ID</b>: %{customdata[0]:i}<br>' +
                      '<b>x</b>: %{customdata[1]:,.2f}<br>' +
                      '<b>y</b>: %{customdata[2]:,.2f}<br>' +
                      '<b>z</b>: %{customdata[3]:,.2f}',
                      showlegend=False)
    
    traces.append(crthit_trace)
    
    fig = go.Figure(traces)
    
    surf_x0 = -210.215*np.ones(100)
    surf_x1 = 210.215*np.ones(100)
    surf_y = np.linspace(-181.6, 134.96, 100)
    surf_z = np.linspace(-894.9505, 894.9505, 100)
    surf_ytest = surf_z
    surf_ztest = surf_y
    #surf_z = np.linspace(232, 432, 100)                                                                                                                   
    rc = np.random.rand(100, 100)
    mycolorscale = [[0, '#aa9ce2'],[1, '#aa9ce2']]
                                                                                                                               
    plane0 = go.Surface(x=surf_x0, y=surf_ytest, z=np.array([surf_ztest] * len(surf_x0)), colorscale=mycolorscale, opacity=0.25, showscale=False, hoverinfo='skip')
    plane1 = go.Surface(x=surf_x1, y=surf_ytest, z=np.array([surf_ztest] * len(surf_x1)), colorscale=mycolorscale, opacity=0.25, showscale=False, hoverinfo='skip')
    
    fig.add_trace(plane0)
    fig.add_trace(plane1)
    fig.update_layout(scene = dict(
        xaxis_title='x [cm]',
        yaxis_title='z [cm]',
        zaxis_title='y [cm]'),
        height=1000, width=1300
    )
    fig.update_layout(margin=dict(l=0, r=0, b=0, t=0))
    fig.update_scenes(aspectmode='data')
    iplot(fig)



In [ ]:
plot_crt_tpc_matches(track_df, crthit_df, matched_track_ids, matched_crthit_ids)